In [2]:
#ssh sydney
#setPython3 

import numpy 
import numpy as np
import math
from numpy import loadtxt
import pandas as pd

from scipy.io import arff
data = arff.loadarff('edm.arff')
df = pd.DataFrame(data[0])

covariables = df.iloc[:,0:16].values
response = df.iloc[:,16:18].values
positions = np.arange(154)

from sklearn.model_selection import train_test_split

covariables_train, covariables_test, response_train, response_test,positions_train,positions_test = train_test_split(covariables, response,positions, test_size=0.20, random_state=42)


##APPRENTISSAGE

def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = numpy.empty(a.shape, dtype=a.dtype)
    shuffled_b = numpy.empty(b.shape, dtype=b.dtype)
    permutation = numpy.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

#import tensorflow as tf 

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


# Import data
# Tensorflow is finicky about shapes, so resize


X_datatrain, X_datatest, targets_train, targets_test = covariables_train, covariables_test, response_train, response_test

#les dimensions de X for training
NbrLignes_train = X_datatrain.shape[0]  ###
NbrColonnes_train= X_datatrain.shape[1] ###

#les dimensions de X for testing
NbrLignes_test = X_datatest.shape[0]  ###
NbrColonnes_test = X_datatest.shape[1] ###
#Y_data = targets 

Y_datatrain = tf.reshape(targets_train, [NbrLignes_train,targets_train.shape[1]])
Y_datatest = tf.reshape(targets_test, [NbrLignes_test,targets_train.shape[1]])

#build the model

# input X: batch_size x NbrColonnes, the first dimension (None) will index the data in the mini-batch
Xfill = tf.placeholder(tf.float32, shape= [None, NbrColonnes_train])
# correct answers will go here
Yfill = tf.placeholder(tf.float32, shape=[None, targets_train.shape[1]]) 
#Let's add some layers

# Probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32)

# five layers and their number of neurons (tha last layer has 10 softmax neurons)
L = 10 #200
M = 2 #160
#N = 100
#O = 30

# Weights initialised with small random values between -0.2 and +0.2
# When using RELUs, make sure biases are initialised with small *positive* values for example 0.1 = tf.ones([K])/10
W1 = tf.Variable(tf.truncated_normal([NbrColonnes_train, L], stddev=0.1))  # 
B1 = tf.Variable(tf.ones([L])/10)
W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
B2 = tf.Variable(tf.ones([M])/10)
#W3 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
#B3 = tf.Variable(tf.ones([N])/10)
#W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
#B4 = tf.Variable(tf.ones([O])/10)
#W5 = tf.Variable(tf.truncated_normal([O, 6], stddev=0.1))
#B5 = tf.Variable(tf.zeros([6]))

# The model, with dropout at each layer


Y1 = tf.nn.relu(tf.matmul(Xfill, W1) + B1)
Y1d = tf.nn.dropout(Y1, pkeep)

#Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + B2)
#Y2d = tf.nn.dropout(Y2, pkeep)

#Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + B3)
#Y3d = tf.nn.dropout(Y3, pkeep)

#Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + B4)
#Y4d = tf.nn.dropout(Y4, pkeep)

#Y_ = tf.matmul(Y4d, W5) + B5


Y_ = tf.matmul(Y1d, W2) + B2

cross_entropy = tf.losses.mean_squared_error(Yfill, Y_)
# accuracy of the trained model, between 0 (worst) and 1 (best)
#Modification de l'accuracy 

#calcul de l'accuracy, une nouvelle manière: :  

# variable learning rate
lr = tf.placeholder(tf.float32)
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy) 

# init
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)


max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 100.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations

train_loss = []
test_loss = []

# Gradient descent loop for 500 steps
for i in range(3500):
 # Select random minibatch
    j = 1
    ##### shuffling our data::: really important
    X_datasf, Y_datasf = shuffle_in_unison(X_datatrain, Y_datatrain.eval())
    
    ##### defining a decreasing learning rate: 
    # learning rate decay
    learning_rate = 0.0001001329829992624
    #learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
    
    ##### 
    for start, end in zip(range(0, NbrLignes_train, 52), range(20, NbrLignes_train, 52)):

             X_batch,Y_batch = X_datasf[start:end,],Y_datasf[start:end,]
             j +=1
             # Do gradient descent step
             _, loss_val_train = sess.run([train_step, cross_entropy], feed_dict={Xfill: X_batch, Yfill: Y_batch, pkeep: 0.75, lr: learning_rate})
             if i %20==0:    
                 
             
                #if j%20==0:
                 loss_val_test = sess.run(cross_entropy, feed_dict={Xfill: X_datatest, Yfill: Y_datatest.eval(), pkeep: 1.0, lr:  learning_rate})
                 train_loss.append(loss_val_train)
                 
                 test_loss.append(loss_val_test)
                 print('LOSS at step %s: - Train loss value : %f -- Test loss value : %f' % (i, loss_val_train,loss_val_test))
#print(test_loss)
#print(test_loss)
#LOSS at step 420: - Train loss value : 22.385496 -- Test loss value : 9.721767

LOSS at step 0: - Train loss value : 0.418338 -- Test loss value : 0.302441
LOSS at step 0: - Train loss value : 0.324999 -- Test loss value : 0.302395
LOSS at step 20: - Train loss value : 0.451582 -- Test loss value : 0.301165
LOSS at step 20: - Train loss value : 0.357303 -- Test loss value : 0.301152
LOSS at step 40: - Train loss value : 0.254216 -- Test loss value : 0.301246
LOSS at step 40: - Train loss value : 0.334239 -- Test loss value : 0.301254
LOSS at step 60: - Train loss value : 0.367973 -- Test loss value : 0.302130
LOSS at step 60: - Train loss value : 0.211268 -- Test loss value : 0.302150
LOSS at step 80: - Train loss value : 0.373455 -- Test loss value : 0.303518
LOSS at step 80: - Train loss value : 0.204125 -- Test loss value : 0.303540
LOSS at step 100: - Train loss value : 0.319951 -- Test loss value : 0.304260
LOSS at step 100: - Train loss value : 0.296266 -- Test loss value : 0.304267
LOSS at step 120: - Train loss value : 0.301519 -- Test loss value : 0.30436

LOSS at step 1100: - Train loss value : 0.222474 -- Test loss value : 0.256960
LOSS at step 1100: - Train loss value : 0.246057 -- Test loss value : 0.256896
LOSS at step 1120: - Train loss value : 0.269341 -- Test loss value : 0.255620
LOSS at step 1120: - Train loss value : 0.256348 -- Test loss value : 0.255548
LOSS at step 1140: - Train loss value : 0.218486 -- Test loss value : 0.252673
LOSS at step 1140: - Train loss value : 0.281894 -- Test loss value : 0.252564
LOSS at step 1160: - Train loss value : 0.204581 -- Test loss value : 0.252355
LOSS at step 1160: - Train loss value : 0.231482 -- Test loss value : 0.252280
LOSS at step 1180: - Train loss value : 0.283342 -- Test loss value : 0.251004
LOSS at step 1180: - Train loss value : 0.178573 -- Test loss value : 0.250913
LOSS at step 1200: - Train loss value : 0.240520 -- Test loss value : 0.250758
LOSS at step 1200: - Train loss value : 0.276499 -- Test loss value : 0.250809
LOSS at step 1220: - Train loss value : 0.261027 -- 

LOSS at step 2160: - Train loss value : 0.182999 -- Test loss value : 0.234954
LOSS at step 2160: - Train loss value : 0.199735 -- Test loss value : 0.235007
LOSS at step 2180: - Train loss value : 0.209420 -- Test loss value : 0.232715
LOSS at step 2180: - Train loss value : 0.216079 -- Test loss value : 0.232725
LOSS at step 2200: - Train loss value : 0.214233 -- Test loss value : 0.233985
LOSS at step 2200: - Train loss value : 0.207392 -- Test loss value : 0.233927
LOSS at step 2220: - Train loss value : 0.172895 -- Test loss value : 0.233506
LOSS at step 2220: - Train loss value : 0.220757 -- Test loss value : 0.233494
LOSS at step 2240: - Train loss value : 0.253472 -- Test loss value : 0.233637
LOSS at step 2240: - Train loss value : 0.215169 -- Test loss value : 0.233608
LOSS at step 2260: - Train loss value : 0.202599 -- Test loss value : 0.231795
LOSS at step 2260: - Train loss value : 0.193915 -- Test loss value : 0.231755
LOSS at step 2280: - Train loss value : 0.183667 -- 

LOSS at step 3220: - Train loss value : 0.193249 -- Test loss value : 0.226459
LOSS at step 3220: - Train loss value : 0.237119 -- Test loss value : 0.226401
LOSS at step 3240: - Train loss value : 0.249140 -- Test loss value : 0.223840
LOSS at step 3240: - Train loss value : 0.174145 -- Test loss value : 0.223796
LOSS at step 3260: - Train loss value : 0.211152 -- Test loss value : 0.226185
LOSS at step 3260: - Train loss value : 0.190529 -- Test loss value : 0.226191
LOSS at step 3280: - Train loss value : 0.187302 -- Test loss value : 0.225738
LOSS at step 3280: - Train loss value : 0.223370 -- Test loss value : 0.225800
LOSS at step 3300: - Train loss value : 0.227646 -- Test loss value : 0.225434
LOSS at step 3300: - Train loss value : 0.201734 -- Test loss value : 0.225518
LOSS at step 3320: - Train loss value : 0.223333 -- Test loss value : 0.224586
LOSS at step 3320: - Train loss value : 0.204442 -- Test loss value : 0.224557
LOSS at step 3340: - Train loss value : 0.184921 -- 

# Let us compute the aRRMSE

In [3]:
learning_rate = 0.0001001329829992624

Y_testpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatest, tf.float32).eval(), Yfill: tf.cast(Y_datatest, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})
#numpy.savetxt('Y_testpred_with_stock_%s_and_ai_%s_modelcut_%s_vartimeframe_%s.txt'%(stock, ai, cut, vartimeframe), Y_testpred, fmt='%f')
Y_trainpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatrain, tf.float32).eval(), Yfill: tf.cast(Y_datatrain, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})


In [4]:
Y_testpred

array([[ 0.01336936, -0.29627833],
       [-0.0222036 , -0.40931678],
       [-0.01517815, -0.3877168 ],
       [ 0.27723137,  0.46779692],
       [ 0.03970774, -0.23299764],
       [ 0.2342838 ,  0.25013196],
       [ 0.2211327 ,  0.27662715],
       [ 0.21348323,  0.27496138],
       [ 0.2598354 ,  0.2917165 ],
       [-0.00603638, -0.35114124],
       [ 0.25159645,  0.38576394],
       [ 0.22891743,  0.31393927],
       [ 0.15962963,  0.10807078],
       [-0.02340811, -0.4194054 ],
       [ 0.2834255 ,  0.4508801 ],
       [-0.01550719, -0.38096148],
       [ 0.03851022, -0.2209176 ],
       [ 0.00666024, -0.27359796],
       [ 0.00658341, -0.28682032],
       [ 0.32136458,  0.5277236 ],
       [ 0.27250588,  0.37393773],
       [ 0.20317493,  0.20584066],
       [ 0.21253932,  0.30614597],
       [ 0.02180384, -0.2776199 ],
       [ 0.2327356 ,  0.32615134],
       [ 0.28014517,  0.40500492],
       [-0.02600013, -0.41755128],
       [ 0.2755431 ,  0.4384333 ],
       [-0.0038654 ,

In [5]:
from numpy import linalg as LA

In [6]:

aRRMSE_list = []

for i in range(Y_testpred.shape[1]) :
    
    Errors_test =  Y_testpred[:,i] - response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    ytrain_mean_i = np.tile(Y_trainpred[:,i].mean(axis = 0), (covariables_test.shape[0], 1))

    Errors_relative = ytrain_mean_i - response_test[:,i]

    Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%Final_SPNNR_aRRMSE)

    aRRMSE_list.append(Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 16.959328
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 14.604560


In [7]:
np.mean(aRRMSE_list)

15.781944266743007

In [9]:
#Taking aRRMSE on PCA or inverted PCA. ?

# Check if old way calculation is wrong

In [10]:
Errors_test =  Y_testpred - response_test


print('<*********************************Let us check the aRRMSE*********************************>')

ytrain_mean = np.tile(Y_trainpred.mean(axis = 0), (response_test.shape[0], 1))

Errors_relative = ytrain_mean - response_test

Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

print("Final_Partition aRRMSE is : %f"%Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final_Partition aRRMSE is : 82.647642


*** The answer is yes ***

# Checking True aRRMSE of S-SPNNR on Slump

In [11]:
spnnr_covariables_train, spnnr_covariables_test, spnnr_response_train, spnnr_response_test,spnnr_positions_train,spnnr_positions_test = train_test_split(covariables, response,positions, test_size=0.33, random_state=42)

In [12]:
spnnr_Y_testpred =np.loadtxt('All_Y_test_predictions.out', delimiter=',' )
spnnr_Y_trainpred =np.loadtxt('All_Y_train_predictions.out', delimiter=',' )

In [10]:
spnnr_aRRMSE_list = []

for i in range(spnnr_Y_testpred.shape[1]) :
    
    spnnr_Errors_test =  spnnr_Y_testpred[:,i] - spnnr_response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    spnnr_ytrain_mean_i = np.tile(spnnr_Y_trainpred[:,i].mean(axis = 0), (spnnr_covariables_test.shape[0], 1))

    spnnr_Errors_relative = spnnr_ytrain_mean_i - spnnr_response_test[:,i]

    spnnr_Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(spnnr_Errors_test)**2)/(LA.norm(spnnr_Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%spnnr_Final_SPNNR_aRRMSE)

    spnnr_aRRMSE_list.append(spnnr_Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 9.212108
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 17.784301
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 18.862991


In [11]:
np.mean(spnnr_aRRMSE_list)

15.286466834678842